<a href="https://colab.research.google.com/github/triagungj/dicoding-ml-beginner-submission/blob/master/Codes/BMLP%20Submission%20Tri%20Agung%20Jiwandono.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Penting**
- Jangan mengubah atau menambahkan cell text yang sudah disediakan, Anda hanya perlu mengerjakan cell code yang sudah disediakan.
- Pastikan seluruh kriteria memiliki output yang sesuai, karena jika tidak ada output dianggap tidak selesai.
- Misal, Anda menggunakan df = df.dropna() silakan gunakan df.isnull().sum() sebagai tanda sudah berhasil. Silakan sesuaikan seluruh output dengan perintah yang sudah disediakan.
- Pastikan Anda melakukan Run All sebelum mengirimkan submission untuk memastikan seluruh cell berjalan dengan baik.
- Pastikan Anda menggunakan variabel df dari awal sampai akhir dan tidak diperbolehkan mengganti nama variabel tersebut.
- Hapus simbol pagar (#) pada kode yang bertipe komentar jika Anda menerapkan kriteria tambahan
- Biarkan simbol pagar (#) jika Anda tidak menerapkan kriteria tambahan
- Pastikan Anda mengerjakan sesuai section yang sudah diberikan tanpa mengubah judul atau header yang disediakan.

# **1. Import Library**
Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [ ]:
import pandas as pd
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import joblib


def resolve_results_path() -> Path:
    """Cari folder Results relatif terhadap notebook."""
    cwd = Path.cwd()
    candidates = [cwd / 'Results', cwd.parent / 'Results']
    for candidate in candidates:
        if candidate.exists():
            return candidate
    raise FileNotFoundError("Folder 'Results' tidak ditemukan dari direktori kerja saat ini.")


def evaluate_model(model, X_test, y_test, average: str = 'weighted'):
    """Hitung metrik utama dan laporkan klasifikasi penuh."""
    y_pred = model.predict(X_test)
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, average=average, zero_division=0),
        'recall': recall_score(y_test, y_pred, average=average, zero_division=0),
        'f1_score': f1_score(y_test, y_pred, average=average, zero_division=0),
    }
    report = classification_report(y_test, y_pred)
    return metrics, report


# **2. Memuat Dataset dari Hasil Clustering**
Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [ ]:
# Memuat dataset hasil clustering yang memiliki label Target
results_dir = resolve_results_path()
data_path = results_dir / 'data_clustering.csv'
df = pd.read_csv(data_path)
print(f'Dataset dimuat dari: {data_path}')
print(f'Banyak baris, kolom: {df.shape}')
print('Distribusi label Target:')
print(df['Target'].value_counts().sort_index())


In [ ]:
# Menampilkan 5 baris pertama untuk memastikan data siap digunakan
df.head()

# **3. Data Splitting**
Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [ ]:
# Menyiapkan fitur (X) dan label (y) kemudian membagi menjadi data latih dan uji
feature_columns = [col for col in df.columns if col != 'Target']
X = df[feature_columns]
y = df['Target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print('Ukuran data latih:', X_train.shape)
print('Ukuran data uji  :', X_test.shape)


# **4. Membangun Model Klasifikasi**
Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Menggunakan algoritma klasifikasi yaitu Decision Tree.
2. Latih model menggunakan data yang sudah dipisah.

In [ ]:
# Melatih model klasifikasi Decision Tree pada data latih
decision_tree_model = DecisionTreeClassifier(random_state=42)
decision_tree_model.fit(X_train, y_train)

dt_metrics, dt_report = evaluate_model(decision_tree_model, X_test, y_test)
print('Metrik evaluasi Decision Tree:')
print(pd.DataFrame([dt_metrics]).round(4))
print('
Laporan klasifikasi Decision Tree:
')
print(dt_report)


In [ ]:
# Menyimpan model Decision Tree terlatih
decision_tree_path = results_dir / 'decision_tree_model.h5'
joblib.dump(decision_tree_model, decision_tree_path)
print(f'Model Decision Tree disimpan ke: {decision_tree_path}')


# **5. Memenuhi Kriteria Skilled dan Advanced dalam Membangun Model Klasifikasi**



**Biarkan kosong jika tidak menerapkan kriteria skilled atau advanced**

In [ ]:
# Melatih model klasifikasi lain untuk eksplorasi (Random Forest)
random_forest_model = RandomForestClassifier(n_estimators=200, random_state=42)
random_forest_model.fit(X_train, y_train)


In [ ]:
# Menampilkan metrik akurasi, presisi, recall, dan F1-score model eksplorasi
rf_metrics, rf_report = evaluate_model(random_forest_model, X_test, y_test)
print('Metrik evaluasi Random Forest:')
print(pd.DataFrame([rf_metrics]).round(4))
print('Laporan klasifikasi Random Forest:')
print(rf_report)


In [ ]:
# Menyimpan model eksplorasi Random Forest
rf_model_path = results_dir / 'explore_random_forest_classification.h5'
joblib.dump(random_forest_model, rf_model_path)
print(f'Model eksplorasi disimpan ke: {rf_model_path}')


Hyperparameter Tuning Model

Pilih salah satu algoritma yang ingin Anda tuning

In [ ]:
# Lakukan Hyperparameter Tuning dan Latih ulang.
# Lakukan dalam satu cell ini saja.
from sklearn.model_selection import GridSearchCV

# Menggunakan Random Forest sebagai kandidat tuning
rf_param_grid = {
    'n_estimators': [120, 160],
    'max_depth': [None, 12],
    'min_samples_split': [2, 4],
}

rf_grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=rf_param_grid,
    cv=3,
    scoring='f1_weighted',
    n_jobs=1,
    verbose=0
)

rf_grid_search.fit(X_train, y_train)

print('Best Params Random Forest:', rf_grid_search.best_params_)
print('Best CV Score:', rf_grid_search.best_score_)

tuning_model = rf_grid_search.best_estimator_

In [ ]:
# Menampilkan hasil evaluasi akurasi, presisi, recall, dan F1-Score pada algoritma yang sudah dituning.
tuning_metrics, tuning_report = evaluate_model(tuning_model, X_test, y_test)
print('Metrik evaluasi model hasil tuning:')
print(pd.DataFrame([tuning_metrics]).round(4))
print('
Laporan klasifikasi model hasil tuning:
')
print(tuning_report)


In [ ]:
# Menyimpan Model hasil tuning
tuning_model_path = results_dir / 'tuning_classification.h5'
joblib.dump(tuning_model, tuning_model_path)
print(f'Model hasil tuning disimpan ke: {tuning_model_path}')


End of Code